# Binary Quadratic Model, Qubo & Ising 

Any qlassf function can be transformed to a binary quadratic model using the `to_bqm` function. In this example we create a generic function that factorize `num`, and we bind `num` with 15.

In [1]:
from qlasskit import qlassf, Qint4, Qint3, Parameter


@qlassf
def test_factor_generic(num: Parameter[Qint4], a: Qint3, b: Qint3) -> Qint4:
    return num - (a * b)


test_factor = test_factor_generic.bind(num=15)
bqm = test_factor.to_bqm()

print("Vars:", bqm.num_variables, "\nInteractions:", bqm.num_interactions)

Vars: 14 
Interactions: 41


A qlassf function can also be exported as a QUBO model using `to_bqm('qubo')` or as an Ising model using `to_bqm('ising')`.


### Running on simulated sampler annealer

Now we can run a simulated sampler annealer to minimize this function; qlasskit offer a `decode_samples` helper function that translates sample result to the high level types of qlasskit.

In [2]:
import neal
from qlasskit.bqm import decode_samples

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)
decoded_samples = decode_samples(test_factor, sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.sample)

{'a': 3, 'b': 5}


The result is 5 and 3 as expected (since 5 times 3 is equal to 15).

### Running on DWave annealer

If we have a valid DWave account, we can run our problem on a real quantum annealer as follow:

In [5]:
from dwave.system import DWaveSampler, EmbeddingComposite

sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=10)
decoded_samples = decode_samples(test_factor, sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.sample)

{'a': 5, 'b': 3}
